In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import sys
sys.path.append("/home/moon/project/prediction-project")

In [9]:
e = pd.read_csv("../data/raw/solar/energy.csv")

power = e["ulsan"].values


In [10]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

e['time'] = e['time'].apply(lambda x:convert_time(x))

In [12]:
def add_random_noise(data, noise_level=0.1, seed=None):
    """
    Add random Gaussian noise to non-zero elements.
    Args:
        data: list or array of data values
        noise_level: fraction of standard deviation relative to value
        seed: random seed for reproducibility
    Returns:
        np.array with noise
    """
    if seed is not None:
        np.random.seed(seed)
    data = np.array(data)
    noise = np.where(data > 0, np.random.normal(0, noise_level * data), 0)
    return np.clip(data + noise, a_min=0, a_max=None)

In [13]:
df = pd.DataFrame()

for i in range(0, 5):
    df_temp = pd.DataFrame()
    df_temp["datetime"] = e["time"]
    df_temp["id"] = i + 1
    add_random_noised_power = add_random_noise(power, noise_level=0.1, seed=i)
    df_temp["power"] = add_random_noised_power
    
    
    df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    


In [14]:
df

,datetime,id,power
0,2018-03-01 0:00:00,1,0.0
1,2018-03-01 1:00:00,1,0.0
2,2018-03-01 2:00:00,1,0.0
3,2018-03-01 3:00:00,1,0.0
4,2018-03-01 4:00:00,1,0.0
...,...,...,...
128155,2021-01-31 19:00:00,5,0.0
128156,2021-01-31 20:00:00,5,0.0
128157,2021-01-31 21:00:00,5,0.0
128158,2021-01-31 22:00:00,5,0.0


In [15]:
df.to_csv("../data/raw/solar/solars.csv", index=False)

In [16]:
w = pd.read_csv("../data/raw/solar/ulsan_obs_data.csv")

In [18]:
w.columns = ['a', 'b', "datetime", "temperature", "wind_speed", "wind_direction", "humidity", "cloud"]

In [20]:
w.drop(columns=["a", "b"], inplace=True)

In [22]:
def convert_cloudy(df: pd.DataFrame, column: str, Forecast: bool = False):
    df = df.copy()
    cloudy = df[column].copy()

    if not Forecast:
        for i in range(len(cloudy)):
            if cloudy.iloc[i] <= 5:
                cloudy.iloc[i] = "Clear"
            elif cloudy.iloc[i] <= 8:
                cloudy.iloc[i] = "Cloudy"
            else:
                cloudy.iloc[i] = "Mostly"
    else:
        for i in range(len(cloudy)):
            if cloudy.iloc[i] <= 2:
                cloudy.iloc[i] = "Clear"
            elif cloudy.iloc[i] <= 3:
                cloudy.iloc[i] = "Cloudy"
            else:
                cloudy.iloc[i] = "Mostly"
    df[column] = cloudy

    return df

In [23]:
w = convert_cloudy(w, "cloud", Forecast=False)

/tmp/ipykernel_210989/2810403767.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mostly' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cloudy.iloc[i] = "Mostly"


In [25]:
w.to_csv("../data/raw/solar/weather.csv", index=False)

In [4]:
l = pd.read_csv("../data/raw/load/loadexample.csv")

In [5]:
load = pd.DataFrame()

In [6]:
load["load"] = l["Load"].values
load["id"] = 1
load["id"] = load["id"].astype(int)
load["load"] = load["load"].astype(float)
load[["id", "load"]].to_csv("../data/raw/load/load.csv", index=False)

In [38]:
l.drop(columns=["Load", "is_hol"], inplace=True)

In [40]:
l.rename(columns={"timestamp": "datetime"}, inplace=True)

In [42]:
l.to_csv("../data/raw/load/weather.csv", index=False)